# Stackexchange

### Introduction

In this lesson, we'll use data from the [stackexchange-postgres](https://github.com/Networks-Learning/stackexchange-dump-to-postgres) repository.

### Connecting to the database with postgres

With postgres, we can connect to the database with the following.

## Connecting with SQLite

> Run if the above did not work.

In [4]:
import sqlite3
conn = sqlite3.connect('stackexchange.db')
cursor = conn.cursor()

In [5]:
import pandas as pd
root_url = "https://raw.githubusercontent.com/sql-fundamentals-jigsaw/mod-1-sql-curriculum/master/5-stackexchange/data/"
names = ['users', 'comments', 'posts', 'votes']
loaded_dfs = [pd.read_csv(f'{root_url}{name}.csv') for name in names]

In [6]:
for index, name in enumerate(names):
    loaded_dfs[index].to_sql(f'{name}', conn, index = False, if_exists = 'replace')

In [7]:
pd.read_sql("""select * from users 
where users.creationdate > '2020-01-01' limit 2""", conn)

,id,reputation,creationdate,displayname,lastaccessdate,websiteurl,location,aboutme,views,upvotes,downvotes,profileimageurl,age,accountid,jsonfield
0,10125,11,2020-03-02 16:43:11.31,David,2020-03-02 17:37:25.6,None,None,None,0,0,0,None,None,17887353,None
1,11428,1,2020-07-03 00:53:08.707,binarystone,2020-07-03 00:53:08.707,None,None,None,0,0,0,https://i.stack.imgur.com/nmcUe.png,None,18972238,None


# Exploring our tables

In [8]:
pd.read_sql("""select * from comments
limit 1""", conn)



,id,postid,score,text,creationdate,userid,jsonfield
0,1,5,0,This is a matter of taste I guess.,2014-01-21 20:34:28.74,10.0,None


In [9]:
pd.read_sql("""select * from posts limit 1""", conn)

,id,posttypeid,acceptedanswerid,parentid,creationdate,score,viewcount,body,owneruserid,lasteditoruserid,...,lasteditdate,lastactivitydate,title,tags,answercount,commentcount,favoritecount,closeddate,communityowneddate,jsonfield
0,1,1,4.0,None,2014-01-21 20:26:05.383,21,2441.0,<p>I was offered a beer the other day that was...,7.0,8.0,...,2014-01-21 22:04:34.977,2014-01-21 22:04:34.977,"What is a citra hop, and how does it differ fr...",<hops>,1.0,0,None,None,None,None


In [10]:
pd.read_sql("""select * from votes limit 1""", conn)

,id,postid,votetypeid,userid,creationdate,bountyamount,jsonfield
0,1,1,2,None,2014-01-21 00:00:00,None,None


We have a couple of key tables.  
* The comments table, which has a postid and a userid.
* The posts table which has an `owneruserid` who made the post, and information like score, viewcount.

Other tables to pay attention to are the posts tables, and the users table.

### Writing our queries

1. Begin by finding the number of users in the database that have a reputation over 100.

In [11]:
pd.read_sql("""select count(id) as number_of_users from users 
where reputation > 100
 """, conn)
# pd.read_sql(query, conn)

# 3566

,number_of_users
0,3566


2. Next find the top five users with the highest average scores, and only include those users who have made more than 10 comments.  Display each user's displayname in the result.

In [12]:
pd.read_sql("""select displayname, avg(comments.score) from users 
join comments on comments.userId = users.id
group by displayname
having count(comments.id) > 10
order by avg(comments.score) desc
limit 5
 """, conn)
# displayname	avg_score
# 0	user23614	2.727273
# 1	wax eagle	1.315789
# 2	user505255	1.307692
# 3	Lucas Kauffman	1.117647
# 4	Fishtoaster	1.047619

,displayname,avg(comments.score)
0,user23614,2.727273
1,wax eagle,1.315789
2,user505255,1.307692
3,Lucas Kauffman,1.117647
4,Fishtoaster,1.047619


3. Next look at the posts table.  Find the `owneruserid`s of those users with the top five average scores, and include the average score.  Only consider those posts where the owneruser created their account after `'2019-01-01'`, and only include posts whose owneruser who had more than five posts. 

In [13]:
pd.read_sql("""select owneruserid, avg(posts.score), users.creationdate from posts
join users on posts.owneruserid = users.id
where users.creationdate > '2019-01-01' 
group by owneruserid
having count(owneruserid) >5
order by avg(posts.score) desc
limit 5
 """, conn)

,owneruserid,avg(posts.score),creationdate
0,8506.0,4.333333,2019-04-01 01:54:54.783
1,8518.0,3.068966,2019-04-05 10:05:29.187
2,14432.0,2.875000,2022-04-21 11:05:00.91
3,11663.0,2.714286,2020-10-26 07:33:59.1
4,8672.0,2.200000,2019-06-25 15:34:28.223


In [15]:
pd.read_sql("""select owneruserid, avg(posts.score)from posts
join users on owneruserid = users.id
where users.creationdate > '2019-01-01' and owneruserid in (select owneruserid from posts group by owneruserid having count(owneruserid) >5)
group by owneruserid
order by avg(posts.score) desc
limit 5
 """, conn)

# 	owneruserid	avg_score
# 0	8506	4.333333
# 1	8518	3.068966
# 2	14432	2.875000
# 3	11663	2.714286
# 4	8672	2.200000

,owneruserid,avg(posts.score)
0,8506.0,4.333333
1,8518.0,3.068966
2,14432.0,2.875000
3,11663.0,2.714286
4,8672.0,2.200000


4. Next find the number of users who have not made a comment.

In [18]:
pd.read_sql("""select sum(case when comments.id is Null then 1 else 0 end) as count from users
left join comments on comments.userid = users.id
""", conn)

# 	count
# 0	8575

,count
0,8575


5. Finally, find the percentage of users who have not made a comment.

In [19]:
pd.read_sql("""select 1.0 * sum(case when comments.id is Null then 1 else 0 end)/count(users.id) as count from users
left join comments on comments.userid = users.id
""", conn)


# users_without_comment
# 0	0.687099


,count
0,0.687099


### Resources

[postgres datasets](https://wiki.postgresql.org/wiki/Sample_Databases)

[google reseearch datasets](https://github.com/google-research-datasets)